<a href="https://colab.research.google.com/github/Rajitha-SL/My-AI-Projects/blob/AI-and-ML-learning/Udacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an example for a typical Convolutional Block

In [1]:
# self.conv1 = nn.Conv2d(3 ,16, 3, padding=1)
# self.pool = nn.MaxPool2d(2,2)
# self.relu1 = nn.ReLU()
# self.drop1 = nn.Dropout2d(0.2)

This is how we do the same with nn.Sequential

In [ ]:
# self.conv_block = nn.Sequential(
#    nn.Conv2d(3, 16, 3, padding = 1),
#    nn.MaxPool(2,2),
#    nn.ReLU(),
#    nn.Dropout(0.2)
# )

This is our first CNN in PyTorch

In [2]:
import torch
import torch.nn as nn

class MyCNN(nn.Module):
  def __init__ (self, n_classes):
    super.__init__()

    # Creating layers. In this case just a standard MLP (Multi Layer Perceptron)
    self.model = nn.Sequential(
        # First conv + maxpool + relu
        nn.Conv2d(3, 16, 3, padding = 1),
        nn.MaxPool(2,2),
        nn.ReLU(),
        nn.Dropout(0.2),

        # Second conv + maxpool + relu
        nn.Conv2d(16, 32, 3, padding = 1),
        nn.MaxPool(2,2),
        nn.ReLU(),
        nn.Dropout(0.2),

        # Third conv + maxpool + relu
        nn.Conv2d(32, 64, 3, padding = 1),
        nn.MaxPool(2,2),
        nn.ReLU(),
        nn.Dropout(0.2),

        # Flatten the 64 feature maps
        nn.Flatten(),

        # Fully connected layers. This assumes
        # that the input image was 32X32
        nn.Linear(1024, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, n_classes)
    )

  def forward(self, x):
    # nn.Sequential will call the layers
    # in the order they have been listed out
    return self.model(x)

